# Chung

In [1]:
from web3 import HTTPProvider
from web3 import Web3
from web3.middleware import geth_poa_middleware
import json
import operator

In [2]:
provider_url = "https://bsc-dataseed4.binance.org/"
web3 = Web3(HTTPProvider(provider_url))
web3.middleware_onion.inject(geth_poa_middleware, layer=0)

print(f"Success?: {web3.isConnected()}")

Success?: True


In [3]:
NUM_BLOCKS_1 = 100
NUM_BLOCKS_2 = 1000

## 1. Số transaction trung bình 100 block

In [4]:
latest_block = web3.eth.block_number
num_txs = 0
for i in range(NUM_BLOCKS_1):
    block_txs = web3.eth.get_block_transaction_count(latest_block - i) 
    num_txs += block_txs
    # print each 10th block to track the progress
    if i % 10 == 0:
        print(f"Block {i} done")
avg_txs = num_txs / NUM_BLOCKS_1

result = {
    'fromBlock': latest_block - NUM_BLOCKS_1 + 1,
    'toBlock': latest_block,
    'averageTransactions': avg_txs
}
json_obj = json.dumps(result, indent=2)
with open('./general1.json', 'w') as f:
    f.write(json_obj)

Block 0 done
Block 10 done
Block 20 done
Block 30 done
Block 40 done
Block 50 done
Block 60 done
Block 70 done
Block 80 done
Block 90 done


## 2. Cho địa chỉ token

In [5]:
token_addr = '0x7083609fCE4d1d8Dc0C979AAb8c869Ea2C873402' 
with open('./abi/erc_20.json', 'r') as f:
    abi = json.load(f)
erc20 = web3.eth.contract(address=token_addr, abi=abi)
latest_block = web3.eth.blockNumber
start_block = latest_block - NUM_BLOCKS_2 + 1
end_block = latest_block
blocks_info = {
    'startBlock': start_block,
    'endBlock': end_block
}
print(f"Start block: {start_block}; End block: {end_block}")

Start block: 21343718; End block: 21344717


In [6]:
# a) Crawl event
transfer_events = erc20.events.Transfer.createFilter(
    fromBlock=start_block, toBlock=end_block).get_all_entries()
transfers = list()
for event in transfer_events:
    transfers.append(json.loads(Web3.toJSON(event)))

result = dict()
result['blocksInfo'] = blocks_info
result['transferEvents'] = transfers
json_obj = json.dumps(result, indent=2)
with open('./general2a.json', 'w') as f:
    f.write(json_obj)

transfers[:2]

[{'args': {'from': '0xe2fc31F816A9b94326492132018C3aEcC4a93aE1',
   'to': '0x2Fa7312f4A2C18D0D4E7AA956F7fE1F11f51f95D',
   'value': 10189646650000000000},
  'event': 'Transfer',
  'logIndex': 6,
  'transactionIndex': 6,
  'transactionHash': '0x8585175fd5d8fb860e9c66054bb4693a3421a626616cd84c7c6b1d057690bc8d',
  'address': '0x7083609fCE4d1d8Dc0C979AAb8c869Ea2C873402',
  'blockHash': '0xe397475842074af208b491c07acc9a6ec188ff2ec59e9231f3e0fb12cdec1df1',
  'blockNumber': 21343732},
 {'args': {'from': '0xcAa0Ee7652be620a8d54FF65C52cCE0e74006FCd',
   'to': '0xDd5bAd8f8b360d76d12FdA230F8BAF42fe0022CF',
   'value': 10000000000000000000},
  'event': 'Transfer',
  'logIndex': 196,
  'transactionIndex': 66,
  'transactionHash': '0xc37295b40249f855344a4902cb1e1317f118c00dc9d939d3accbc5b983c114b4',
  'address': '0x7083609fCE4d1d8Dc0C979AAb8c869Ea2C873402',
  'blockHash': '0xb20b6f7dc48c1c289abb5644cc08b7ccac8023042034629d64482019e0f8d361',
  'blockNumber': 21343794}]

In [7]:
# b) decimals
decimals = erc20.functions.decimals().call()
print(f"Token decimals: {decimals}")

result = {
    'decimals': decimals,
}
json_obj = json.dumps(result, indent=2)
with open('./general2b.json', 'w') as f:
    f.write(json_obj)

Token decimals: 18


In [8]:
# c) supply
token_total_supply = erc20.functions.totalSupply().call() / 10**decimals
print(f"Token supply: {token_total_supply}")

result = {
    'totalSupply': token_total_supply,
}
json_obj = json.dumps(result, indent=2)
with open('./general2c.json', 'w') as f:
    f.write(json_obj)

Token supply: 15999999.991309563


In [9]:
# d) user with the most transactions from
from_wallet_transfers = dict()
for transfer in transfers:
    from_wallet = transfer['args']['from']
    if from_wallet not in from_wallet_transfers:
        from_wallet_transfers[from_wallet] = 1
    else:
        from_wallet_transfers[from_wallet] += 1
# max from_wallet addr
max_from_wallet, max_frequency = max(from_wallet_transfers.items(), key=operator.itemgetter(1))
balance = erc20.functions.balanceOf(max_from_wallet).call() / 10**decimals

print(f"Most transactions from {max_from_wallet} with balance: {balance}")

result = dict()
result['blocksInfo'] = blocks_info
result.update({
    'wallet': max_from_wallet,
    'maxFrequency': max_frequency,
    'balance': balance
})
json_obj = json.dumps(result, indent=2)
with open('./general2d.json', 'w') as f:
    f.write(json_obj)

Most transactions from 0xAF1965202EdD913d729Bc4C5608d69F23eB66f26 with balance: 27038.460063948056


In [10]:
# e) user with the most transacitons to
to_wallet_transfers = dict()
for transfer in transfers:
    to_wallet = transfer['args']['to']
    if to_wallet not in to_wallet_transfers:
        to_wallet_transfers[to_wallet] = 1
    else:
        to_wallet_transfers[to_wallet] += 1

max_to_wallet, max_frequency = max(to_wallet_transfers.items(), key=operator.itemgetter(1))
balance = erc20.functions.balanceOf(max_to_wallet).call() / 10**decimals

print(f"Most transactions to {max_to_wallet} with balance: {balance}")

result = dict()
result['blocksInfo'] = blocks_info
result.update({
    'wallet': max_to_wallet,
    'max_frequency': max_frequency,
    'balance': balance,
})
json_obj = json.dumps(result, indent=2)
with open('./general2e.json', 'w') as f:
    f.write(json_obj)

Most transactions to 0xDd5bAd8f8b360d76d12FdA230F8BAF42fe0022CF with balance: 532978.7203801092


In [11]:
# f) symbol + name
symbol = erc20.functions.symbol().call()
name = erc20.functions.name().call()
print(f"Symbol: {symbol}; Name: {name}")

result = {
    'symbol': symbol,
    'name': name
}

json_obj = json.dumps(result, indent=2)
with open('./general2f.json', 'w') as f:
    f.write(json_obj)

Symbol: DOT; Name: Polkadot Token
